In [1]:
# mount drive, to use the uploaded dataset

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# install dependencies

%%capture
!pip install tensorboardX

In [3]:
# clone code

!git clone https://github.com/avinashpaliwal/Super-SloMo.git
%cd Super-SloMo

Cloning into 'Super-SloMo'...
remote: Enumerating objects: 174, done.
remote: Total 174 (delta 0), reused 0 (delta 0), pack-reused 174
Receiving objects: 100% (174/174), 26.88 MiB | 34.49 MiB/s, done.
Resolving deltas: 100% (93/93), done.
/content/Super-SloMo


In [0]:
# get dataset

%%capture
!rm -rf val
!cp '/content/drive/My Drive/__progetto int sys/data/val - Star360.zip' './'
!mkdir val
!unzip 'val - Star360.zip' -d val

In [0]:
# inference for images

from time import time
import click
import cv2
import torch
from PIL import Image
import numpy as np
import model
from torchvision import transforms
from torch.functional import F
import os

torch.set_grad_enabled(False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trans_forward = transforms.ToTensor()
trans_backward = transforms.ToPILImage()
if device != "cpu":
    mean = [0.429, 0.431, 0.397]
    mea0 = [-m for m in mean]
    std = [1] * 3
    trans_forward = transforms.Compose([trans_forward, transforms.Normalize(mean=mean, std=std)])
    trans_backward = transforms.Compose([transforms.Normalize(mean=mea0, std=std), trans_backward])

flow = model.UNet(6, 4).to(device)
interp = model.UNet(20, 5).to(device)
back_warp = None


def setup_back_warp(w, h):
    global back_warp
    with torch.set_grad_enabled(False):
        back_warp = model.backWarp(w, h, device).to(device)


def load_models(checkpoint):
    states = torch.load(checkpoint, map_location='cpu')
    interp.load_state_dict(states['state_dictAT'])
    flow.load_state_dict(states['state_dictFC'])


def interpolate_batch(frames, factor):
    frame0 = torch.stack(frames[:-1])
    frame1 = torch.stack(frames[1:])

    i0 = frame0.to(device)
    i1 = frame1.to(device)
    ix = torch.cat([i0, i1], dim=1)

    flow_out = flow(ix)
    f01 = flow_out[:, :2, :, :]
    f10 = flow_out[:, 2:, :, :]

    frame_buffer = []
    for i in range(1, factor):
        t = i / factor
        temp = -t * (1 - t)
        co_eff = [temp, t * t, (1 - t) * (1 - t), temp]

        ft0 = co_eff[0] * f01 + co_eff[1] * f10
        ft1 = co_eff[2] * f01 + co_eff[3] * f10

        gi0ft0 = back_warp(i0, ft0)
        gi1ft1 = back_warp(i1, ft1)

        iy = torch.cat((i0, i1, f01, f10, ft1, ft0, gi1ft1, gi0ft0), dim=1)
        io = interp(iy)

        ft0f = io[:, :2, :, :] + ft0
        ft1f = io[:, 2:4, :, :] + ft1
        vt0 = F.sigmoid(io[:, 4:5, :, :])
        vt1 = 1 - vt0

        gi0ft0f = back_warp(i0, ft0f)
        gi1ft1f = back_warp(i1, ft1f)

        co_eff = [1 - t, t]

        ft_p = (co_eff[0] * vt0 * gi0ft0f + co_eff[1] * vt1 * gi1ft1f) / \
               (co_eff[0] * vt0 + co_eff[1] * vt1)

        frame_buffer.append(ft_p)

    return frame_buffer


def denorm_frame(frame, w0, h0):
    frame = frame.cpu()
    frame = trans_backward(frame)
    frame = frame.resize((w0, h0), Image.BILINEAR)
    frame = frame.convert('RGB')
    return np.array(frame)[:, :, ::-1].copy()


def interpolate_pics(source, dest, factor, batch_size=10):
    img = []
    for i in range(2):
        img.append(cv2.imread(source[i]))
    h0 = img[0].shape[0]
    w0 = img[0].shape[1]

    w, h = (w0 // 32) * 32, (h0 // 32) * 32
    setup_back_warp(w, h)

    batch = []
    for frame in img:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        frame = frame.resize((w, h), Image.ANTIALIAS)
        frame = frame.convert('RGB')
        frame = trans_forward(frame)
        batch.append(frame)

    intermediate_frames = interpolate_batch(batch, factor)
    intermediate_frames = list(zip(*intermediate_frames))
    
    i = 0
    for fid, iframe in enumerate(intermediate_frames):
        for frm in iframe:
            cv2.imwrite(os.path.join(dest, str(i) + '.png'), denorm_frame(frm, w0, h0))
            i += 1

def main(input, checkpoint, output_folder, batch = 2, scale = 2):
    avg = lambda x, n, x0: (x * n/(n+1) + x0 / (n+1), n+1)
    load_models(checkpoint)
    interpolate_pics(input, output_folder, int(scale), int(batch))

In [0]:
# get pretrained model
# you can get it here: https://drive.google.com/open?id=1IvobLDbRiBgZr3ryCRrWL8xDbMZ-KnpF

!cp '/content/drive/My Drive/__progetto int sys/ckps/SuperSloMo.ckpt' 'data/SuperSloMo.ckpt'

In [14]:
# generate interpolated images

!rm -rf ./interp
!mkdir ./interp

import os

basepath = './val'
sub_f = os.listdir(basepath)
sub_f.sort()
for s in sub_f:
  path = os.path.join(basepath, s)
  file1 = path + '/0.jpg'
  file2 = path + '/2.jpg'
  os.makedirs(os.path.join('./interp', s))
  main(input = [file1, file2], checkpoint = 'data/SuperSloMo.ckpt', output_folder = os.path.join('./interp', s))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2506: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2705: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [15]:
# get stats

import PIL
import PIL.Image
import skimage
import skimage.measure
import numpy
import math

gt = '/1.jpg'
inter = '/0.png'

basepath1 = './val'
basepath2 = './interp'
sub_f = os.listdir(basepath)
sub_f.sort()
dblPsnr = []
dblSsim = []
dblIE = []
for s in sub_f:
  path1 = os.path.join(basepath1, s)
  path2 = os.path.join(basepath2, s)
  dblPsnr.append(skimage.measure.compare_psnr(im_true=numpy.array(PIL.Image.open(path1 + gt))[:, :, ::-1], im_test=numpy.array(PIL.Image.open(path2 + inter))[:, :, ::-1], data_range=255))
  dblSsim.append(skimage.measure.compare_ssim(X=numpy.array(PIL.Image.open(path1 + gt))[:, :, ::-1], Y=numpy.array(PIL.Image.open(path2 + inter))[:, :, ::-1], data_range=255, multichannel=True))
  dblIE.append(math.sqrt(skimage.measure.compare_mse(numpy.array(PIL.Image.open(path1 + gt))[:, :, ::-1], numpy.array(PIL.Image.open(path2 + inter))[:, :, ::-1])))

print('computed average psnr', numpy.mean(dblPsnr))
print('computed average ssim', numpy.mean(dblSsim))
print('computed average ie', numpy.mean(dblIE))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: DEPRECATED: skimage.measure.compare_psnr has been moved to skimage.metrics.peak_signal_noise_ratio. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: DEPRECATED: skimage.measure.compare_mse has been moved to skimage.metrics.mean_squared_error. It will be removed from skimage.measure in version 0.18.


computed average psnr 25.076035286598195
computed average ssim 0.895374701408127
computed average ie 16.413606068514156
